In [1]:
import openpyxl

In [2]:
wb = openpyxl.Workbook()

In [2]:
wb = openpyxl.load_workbook('test_py_styles.xlsx')

In [16]:
new_work_sheet = wb.get_sheet_by_name("ParsedDbs_wparse_Oct")

In [3]:
new_work_sheet = wb.get_sheet_by_name("ParsedOcbc_wparse_Oct")

In [21]:
new_work_sheet = wb.create_sheet()
new_work_sheet.title = "ParsedDbs_wparse_Oct"
#new_work_sheet.title = "ParsedOcbc_wparse_Oct"
#new_work_sheet.title = "Parsed_wparse_Oct"

# PROCESSINGS

In [5]:
from collections import namedtuple
import yaml

tuple_wlist = namedtuple('WList', ['word_list', 'description', 'category'])
fo = open('expense_parsing_wordlist.yaml')
fo_str = fo.read()
fo.close()
wlist = yaml.load(fo_str)

wlist_dbs = []
wlist_ocbc = []

for mat in wlist['DBS']:
    mat_tupled = tuple_wlist(**mat)
    wlist_dbs.append(mat_tupled)
for mat in wlist['OCBC']:
    mat_tupled = tuple_wlist(**mat)
    wlist_ocbc.append(mat_tupled)

In [6]:
import re
def parseCategory(str,wlist):
    for entry in wlist:
        for word in entry.word_list:
            if re.search(word,str):
                return entry.category, entry.description

In [ ]:
import re

def valid_date_DBS(datestring):
        try:
                mat=re.match('^((([0-9])|([0-2][0-9])|([3][0-1]))\ (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\ \d{4})|(T\+[0-9]+)$', datestring)
                #mat=re.match('^[01][0-9] (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{4}$', datestring)
                if mat is not None:
                        return True
        except: # ValueError:
                pass
        return False

In [ ]:
def get_first(iterable, default=None):
    if iterable:
        for item in iterable:
            return item
    return default

# DBS

In [17]:
import re

def valid_date_DBS(datestring):
        try:
                mat=re.match('^((([0-9])|([0-2][0-9])|([3][0-1]))\ (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\ \d{4})|(T\+[0-9]+)$', datestring)
                #mat=re.match('^[01][0-9] (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{4}$', datestring)
                if mat is not None:
                        return True
        except: # ValueError:
                pass
        return False

In [18]:
def get_first(iterable, default=None):
    if iterable:
        for item in iterable:
            return item
    return default

In [19]:
import csv
#f = open('acb5e3dd9227f4b749d58dc810ed7059.P000000011915007.csv', 'rt')
#f = open('dbs_2015_0510_2709.csv', 'rt')
#f = open('dbs_2015_1810_1210.csv', 'rt')
f = open('dbs_latest.csv', 'rt')
old_line = []
filtered_lines = []
saldo_akhir_dbs_from_bank = 0
saldo_ledger_dbs_from_bank = 0
try:
    reader = csv.reader(f)
    for line in reader:
        
        if (len(line) == 2) and line[0] == 'Available Balance:':
            _saldo_dbs = float(line[1].replace(',',''))
            print 'got saldo ', _saldo_dbs
            saldo_akhir_dbs_from_bank = _saldo_dbs
            
        if (len(line) == 2) and line[0] == 'Ledger Balance:':
            _saldo_dbs = float(line[1].replace(',',''))
            print 'got saldo ', _saldo_dbs
            saldo_ledger_dbs_from_bank = _saldo_dbs
        
        if valid_date_DBS(get_first(line)):
            #print old_line
            #old_line = line
            #print line
            filtered_lines.append(line)
        #else:
            #old_line= old_line + line
finally:
    f.close()
    for item in filtered_lines:
        print item

got saldo  1211.05
got saldo  1224.85
['08 Nov 2015', 'POS', ' 13.80', ' ', 'NTUC FP', '', '', '']
['08 Nov 2015', 'POS', ' 32.96', ' ', 'RE & S ENT', '', '', '']
['05 Nov 2015', 'MST', ' 10.60', ' ', 'LAZADA SINGAPORE PTE L SI NG 01NOV', '5548-2740-7025-2266', '', '']
['05 Nov 2015', 'MST', ' 48.60', ' ', 'WWF-DONATION           SI NG 04NOV', '5548-2740-7025-2266', '', '']
['05 Nov 2015', 'AWL', ' 100.00', ' ', '70252266', 'PIONEER MRT 1', '', '', '']
['04 Nov 2015', 'POS', ' 20.00', ' ', 'TRANSIT LI', '', '', '']
['04 Nov 2015', 'MST', ' 9.14', ' ', 'PAYPAL *NETFLIX.COM    40 29 02NOV', '5548-2740-7025-2266', '', '']
['04 Nov 2015', 'MST', ' 90.75', ' ', 'TIGERAIR    H4MZ3C     SI NG 01NOV', '5548-2740-7025-2266', '', '']
['03 Nov 2015', 'MST', ' 15.00', ' ', 'AXS PAYMENT            SI NG 02NOV', '5548-2740-7025-2266', '', '']


In [20]:
import datetime
import pytz
from collections import namedtuple

tz =  pytz.timezone('Asia/Singapore')

from datetime import datetime

from openpyxl.compat import range
row = 9

parsed_dbs = []
tuple_parsed_dbs = namedtuple('ParsedDbs', ['when', 'tval', 'info_string'])

for item in filtered_lines:
    #print get_first(item).split()[0:3]
    day, month, year = get_first(item).split()[0:3]
    _when = datetime.strptime("{yyyy} {Mmm} {dd}".format(yyyy=year, Mmm=month, dd=day), "%Y %b %d")
    if not _when.tzinfo:
      _when = tz.localize(_when)
    info_string = ''
    #for info in item[2:3]:
    try:
        _tval = -float(item[2])
    except:
        _tval = float(item[3])
    
    info_string = item[1] + ' ' + item[4]
    for info in item[5::]:
        info_string = info_string + ' ' + info
    print _when, _tval, info_string
    _item_tupled = tuple_parsed_dbs(_when, _tval, info_string)
    parsed_dbs.append(_item_tupled)
    #parsed_dbs.append([_when, _tval, info_string])
    #sheet.cell(column=1,row=row).value = _tval
    row = row + 1

2015-11-08 00:00:00+08:00 -13.8 POS NTUC FP   
2015-11-08 00:00:00+08:00 -32.96 POS RE & S ENT   
2015-11-05 00:00:00+08:00 -10.6 MST LAZADA SINGAPORE PTE L SI NG 01NOV 5548-2740-7025-2266  
2015-11-05 00:00:00+08:00 -48.6 MST WWF-DONATION           SI NG 04NOV 5548-2740-7025-2266  
2015-11-05 00:00:00+08:00 -100.0 AWL 70252266 PIONEER MRT 1   
2015-11-04 00:00:00+08:00 -20.0 POS TRANSIT LI   
2015-11-04 00:00:00+08:00 -9.14 MST PAYPAL *NETFLIX.COM    40 29 02NOV 5548-2740-7025-2266  
2015-11-04 00:00:00+08:00 -90.75 MST TIGERAIR    H4MZ3C     SI NG 01NOV 5548-2740-7025-2266  
2015-11-03 00:00:00+08:00 -15.0 MST AXS PAYMENT            SI NG 02NOV 5548-2740-7025-2266  


## Process Dbs

In [24]:
#parsed_dbs_filtered_by_date = [x for x in parsed_dbs if x.when > datetime(2015, 8, 24, tzinfo=tz)]
parsed_dbs_filtered_by_date = parsed_dbs
#parsed_dbs_filtered_by_date = [x for x in parsed_dbs if x.when < datetime(2015, 10, 5, tzinfo=tz)]
#parsed_dbs_filtered_by_date = [x for x in parsed_dbs if x.when.day < 5 or x.when.month < 10]

from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Style, Color
from openpyxl.cell import coordinate_from_string, column_index_from_string, get_column_letter

row = 2
index_row = 0
saldo_awal = 1565.7 #40.6 #1056.19 # 1243.82 # 1490.13#342.35 # 911.05 # 841.14
saldo_akhir = saldo_awal
sheet = new_work_sheet

category_set_weekly_dbs  = {}
category_set_monthly_dbs = {}
category_set_detailed_monthly_dbs = {}

sheet.cell(column=2,row=1).value = saldo_awal

for parsed_item in reversed(parsed_dbs_filtered_by_date):
    sheet.cell(column=2,row=row).value = '=' + get_column_letter(2) + str(row-1) + '+' + get_column_letter(4) + str(row)
    #_cell = sheet.cell(column=3,row=row)
    #_cell = parsed_dbs[index_row][0]
    sheet[get_column_letter(1) + str(row)] = parsed_item.when
    #_cell.style.number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    sheet[get_column_letter(1) + str(row)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    #saldo = saldo - sheet_data.cell(column=1,row=row).value 
    sheet[get_column_letter(4) + str(row)].value = parsed_item.tval
    sheet[get_column_letter(5) + str(row)].value = parsed_item.info_string
    
    category = parseCategory(parsed_item.info_string, wlist_dbs)
    #if category:
    #    if len(category[0])>1:
    #        sheet[get_column_letter(5) + str(row)].value = 'MULTIPLE: '+';'.join(category[0])
    #    else:
    #        sheet[get_column_letter(5) + str(row)].value = ';'.join(category[0])
    #    sheet[get_column_letter(6) + str(row)].value = ';'.join(category[1])
    #else:
    #    sheet[get_column_letter(5) + str(row)].value = 'NONE'
    
    if category:
        sheet[get_column_letter(6) + str(row)].value = category[0]['detailed_monthly']
        sheet[get_column_letter(7) + str(row)].value = category[0]['monthly']
        sheet[get_column_letter(8) + str(row)].value = category[0]['weekly']
        sheet[get_column_letter(3) + str(row)].value = ';'.join(category[1])
        category_set_monthly_dbs[category[0]['monthly']] = category_set_monthly_dbs.get(category[0]['monthly'],0)+parsed_item.tval
        category_set_weekly_dbs[category[0]['weekly']] = category_set_weekly_dbs.get(category[0]['weekly'],0)+parsed_item.tval
        category_set_detailed_monthly_dbs[category[0]['detailed_monthly']] = category_set_detailed_monthly_dbs.get(category[0]['detailed_monthly'],0)+parsed_item.tval
    else:
        sheet[get_column_letter(6) + str(row)].value = 'PLEASE_DESC'
        sheet[get_column_letter(7) + str(row)].value = 'PLEASE_DESC'
        sheet[get_column_letter(8) + str(row)].value = 'PLEASE_DESC'
    
    saldo_akhir = saldo_akhir + parsed_item.tval
    row=row+1
    index_row = index_row + 1

sheet.column_dimensions['A'].width = 35

print saldo_akhir, saldo_akhir_dbs_from_bank, saldo_ledger_dbs_from_bank
if 0.01 < abs(saldo_akhir - saldo_akhir_dbs_from_bank):
    print "WARNING: saldo not tally!"
    if 0.01 > abs(saldo_akhir - saldo_ledger_dbs_from_bank):
        print "INFERENCE: Possible unsettled transaction in Debit Card detected..."
    print "CHECK: \n- if you got any unsettled transaction (e.g. debit cards) \n- or transaction date cut-off is wrong \n- or saldo is incorrect"
    print '(Total) Difference is ', -saldo_akhir + saldo_akhir_dbs_from_bank
print -saldo_akhir + saldo_akhir_dbs_from_bank, -saldo_akhir + saldo_ledger_dbs_from_bank

1224.85 1211.05 1224.85
INFERENCE: Possible unsettled transaction in Debit Card detected...
CHECK: 
- if you got any unsettled transaction (e.g. debit cards) 
- or transaction date cut-off is wrong 
- or saldo is incorrect
(Total) Difference is  -13.8
-13.8 -2.27373675443e-13


In [25]:
print category_set_monthly_dbs, sum(category_set_monthly_dbs.values())
print category_set_weekly_dbs, sum(category_set_weekly_dbs.values())
print category_set_detailed_monthly_dbs, sum(category_set_detailed_monthly_dbs.values())

{'giving': -48.6, 'makan_atm': -100.0, 'recurr': -9.14, 'transport': -20.0, 'makan': -13.8} -191.54
{'NONE': -157.74, 'groceries': -13.8, 'transport': -20.0} -191.54
{'giving': -48.6, 'groceries': -13.8, 'NONE': -100.0, 'online_subscription': -9.14, 'transport': -20.0} -191.54


In [26]:
wb.save('test_py_styles.xlsx')

# OCBC

In [11]:
import datetime
import re

def valid_date(datestring):
        try:
                mat=re.match('(\d{1,2})[/.-](\d{1,2})[/.-](\d{4})$', datestring)
                if mat is not None:
                        datetime.datetime(*(map(int, mat.groups()[-1::-1])))
                        return True
        except: # ValueError:
                pass
        return False

In [12]:
import csv
#f = open('TransactionHistory_20150920092712.csv', 'rt')
#f = open('TransactionHistory_2015_0510_2709.csv', 'rt')
#f = open('TransactionHistory_2015_1810_1210.csv', 'rt')
f = open('ocbc_latest.csv', 'rt')
old_line = []
filtered_lines = []
g_first = True

saldo_akhir_ocbc_from_bank = 0
saldo_ledger_ocbc_from_bank = 0

try:
    reader = csv.reader(f)
    for line in reader:
        
        if (len(line) == 2) and line[0] == 'Available ':
            _saldo_ocbc = float(line[1].replace(',',''))
            print 'got saldo ', _saldo_ocbc
            saldo_akhir_ocbc_from_bank = _saldo_ocbc
        
        if (len(line) == 2) and line[0] == 'Balance':
            _saldo_ocbc = float(line[1].replace(',',''))
            print 'got saldo ', _saldo_ocbc
            saldo_ledger_ocbc_from_bank = _saldo_ocbc
        #print line, get_first(line), valid_date(get_first(line))
        if valid_date(get_first(line)):
            #print old_line
            if (False == g_first):
                filtered_lines.append(old_line)
            else:
                g_first = False    
            old_line = line
        else:
            old_line= old_line + line
    filtered_lines.append(old_line)            
finally:
    f.close()
    for item in filtered_lines:
        print item

got saldo  3996.82
got saldo  3996.82
['07/11/2015', '07/11/2015', 'PURCHASE        NETS', '30.00', '', '', '', 'THANK YOU                PAYA LEBA  TRANSIT LI ']
['06/11/2015', '06/11/2015', 'CASH WITHDRAWAL  ATM', '100.00', '', '', '', 'OCBC-AMOY ST FOOD CENTRE SINGAPORE']
['02/11/2015', '02/11/2015', 'CASH WITHDRAWAL  ATM', '100.00', '', '', '', 'OCBC-AMOY ST FOOD CENTRE SINGAPORE']
['02/11/2015', '02/11/2015', 'IBG GIRO', '462.68', '', '', '', '88854193 *PRUDENTIAL A CO NO COLL 88854193 ']
['02/11/2015', '02/11/2015', 'IBG GIRO', '405.93', '', '', '', '88854193 *PRUDENTIAL A CO NO COLL 88854193 ']


In [13]:
import datetime
import pytz
from collections import namedtuple

tz =  pytz.timezone('Asia/Singapore')

from datetime import datetime

parsed_ocbc = []
tuple_parsed_ocbc = namedtuple('ParsedOcbc', ['when', 'when_value', 'tval', 'info_string'])

for item in filtered_lines:
    #print get_first(item)
    #_when = get_valid_date(get_first(item))
    _when = datetime.strptime(get_first(item), "%d/%m/%Y")
    _when2 = datetime.strptime(item[1], "%d/%m/%Y")
    if not _when.tzinfo:
      _when = tz.localize(_when)
    if not _when2.tzinfo:
      _when2 = tz.localize(_when2)
    info_string = ''
    #for info in item[2::]:
    try:
        _tval = -float(item[3].replace(',',''))
    except:
        _tval = float(item[4].replace(',',''))
    
    if len(item) > 5:
        info_string = item[2] + ' ' + item[5]
        for info in item[6::]:
            info_string = info_string + ' ' + info
    else:
        info_string = item[2]
    print _when , _when2, _tval, info_string
    _item_tupled = tuple_parsed_ocbc(_when , _when2, _tval, info_string)
    parsed_ocbc.append(_item_tupled)

2015-11-07 00:00:00+08:00 2015-11-07 00:00:00+08:00 -30.0 PURCHASE        NETS   THANK YOU                PAYA LEBA  TRANSIT LI 
2015-11-06 00:00:00+08:00 2015-11-06 00:00:00+08:00 -100.0 CASH WITHDRAWAL  ATM   OCBC-AMOY ST FOOD CENTRE SINGAPORE
2015-11-02 00:00:00+08:00 2015-11-02 00:00:00+08:00 -100.0 CASH WITHDRAWAL  ATM   OCBC-AMOY ST FOOD CENTRE SINGAPORE
2015-11-02 00:00:00+08:00 2015-11-02 00:00:00+08:00 -462.68 IBG GIRO   88854193 *PRUDENTIAL A CO NO COLL 88854193 
2015-11-02 00:00:00+08:00 2015-11-02 00:00:00+08:00 -405.93 IBG GIRO   88854193 *PRUDENTIAL A CO NO COLL 88854193 


## Process Ocbc

In [14]:
#parsed_ocbc_filtered_by_date = [x for x in parsed_ocbc if x.when > datetime(2015, 8, 23, tzinfo=tz)]
#parsed_ocbc_filtered_by_date = [x for x in parsed_ocbc if x.when < datetime(2015, 10, 5, tzinfo=tz)]
#parsed_ocbc_filtered_by_date = [x for x in parsed_ocbc if x.when.day < 5 or x.when.month < 10]
parsed_ocbc_filtered_by_date = parsed_ocbc
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Style, Color
from openpyxl.cell import coordinate_from_string, column_index_from_string, get_column_letter

row = 2 #23 #2
index_row = 0
saldo_awal = 5095.43 #7842.91 #4537.36 #4637.36 #9425.14 # 4342.14 # 5632.2
saldo_akhir = saldo_awal
sheet = new_work_sheet

category_set_weekly_ocbc  = {}
category_set_monthly_ocbc = {}
category_set_detailed_monthly_ocbc = {}

sheet.cell(column=2,row=1).value = saldo_awal

for parsed_item in reversed(parsed_ocbc_filtered_by_date):
    sheet.cell(column=2,row=row).value = '=' + get_column_letter(2) + str(row-1) + '+' + get_column_letter(4) + str(row)
    #_cell = sheet.cell(column=3,row=row)
    #_cell = parsed_dbs[index_row][0]
    sheet[get_column_letter(1) + str(row)] = parsed_item.when
    #_cell.style.number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    sheet[get_column_letter(1) + str(row)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    #saldo = saldo - sheet_data.cell(column=1,row=row).value 
    sheet[get_column_letter(4) + str(row)].value = parsed_item.tval
    sheet[get_column_letter(5) + str(row)].value = parsed_item.info_string
    
    category = parseCategory(parsed_item.info_string, wlist_ocbc)
    #if category:
    #    if len(category[0])>1:
    #        sheet[get_column_letter(5) + str(row)].value = 'MULTIPLE: '+';'.join(category[0])
    #    else:
    #        sheet[get_column_letter(5) + str(row)].value = ';'.join(category[0])
    #    sheet[get_column_letter(6) + str(row)].value = ';'.join(category[1])
    #else:
    #    sheet[get_column_letter(5) + str(row)].value = 'NONE'
    
    if category:
        sheet[get_column_letter(6) + str(row)].value = category[0]['detailed_monthly']
        sheet[get_column_letter(7) + str(row)].value = category[0]['monthly']
        sheet[get_column_letter(8) + str(row)].value = category[0]['weekly']
        sheet[get_column_letter(3) + str(row)].value = ';'.join(category[1])
        category_set_monthly_ocbc[category[0]['monthly']] = category_set_monthly_ocbc.get(category[0]['monthly'],0)+parsed_item.tval
        category_set_weekly_ocbc[category[0]['weekly']] = category_set_weekly_ocbc.get(category[0]['weekly'],0)+parsed_item.tval
        category_set_detailed_monthly_ocbc[category[0]['detailed_monthly']] = category_set_detailed_monthly_ocbc.get(category[0]['detailed_monthly'],0)+parsed_item.tval
    else:
        sheet[get_column_letter(6) + str(row)].value = 'PLEASE_DESC'
        sheet[get_column_letter(7) + str(row)].value = 'PLEASE_DESC'
        sheet[get_column_letter(8) + str(row)].value = 'PLEASE_DESC'
    
    saldo_akhir = saldo_akhir + parsed_item.tval
    row=row+1
    index_row = index_row + 1

sheet.column_dimensions['A'].width = 35

print saldo_akhir, saldo_akhir_ocbc_from_bank, saldo_ledger_ocbc_from_bank
if 0.01 < abs(saldo_akhir - saldo_akhir_ocbc_from_bank):
    print "WARNING: saldo not tally!"
    if 0.01 > abs(saldo_akhir - saldo_ledger_ocbc_from_bank):
        print "INFERENCE: Possible unsettled transaction in Debit Card detected..."
    print "CHECK: \n- if you got any unsettled transaction (e.g. debit cards) \n- or transaction date cut-off is wrong \n- or saldo is incorrect"
    print '(Total) Difference is ', -saldo_akhir + saldo_akhir_ocbc_from_bank
else:
    print 'OK, saldo tally.'
print -saldo_akhir + saldo_akhir_ocbc_from_bank, -saldo_akhir + saldo_ledger_ocbc_from_bank

3996.82 3996.82 3996.82
OK, saldo tally.
4.54747350886e-13 4.54747350886e-13


In [15]:
print category_set_monthly_ocbc, sum(category_set_monthly_ocbc.values())
print category_set_weekly_ocbc, sum(category_set_weekly_ocbc.values())
print category_set_detailed_monthly_ocbc, sum(category_set_detailed_monthly_ocbc.values())

{'makan_atm': -200.0, 'insurance': -868.61, 'transport': -30.0} -1098.61
{'NONE': -1068.6100000000001, 'transport': -30.0} -1098.61
{'NONE': -200.0, 'insurance': -868.61, 'ezlink': -30.0} -1098.61


In [16]:
wb.save('test_py_styles.xlsx')

In [2]:
wbmain = openpyxl.load_workbook('C:\Users\Billy\Dropbox\Marriage\FinancialPlan\Budget-latest_(LibreOffice)_uv.xlsx')

C:\Python27\lib\site-packages\openpyxl\workbook\names\named_range.py:121: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


In [3]:
wbmain_data = openpyxl.load_workbook('C:\Users\Billy\Dropbox\Marriage\FinancialPlan\Budget-latest_(LibreOffice)_uv.xlsx',data_only=True)

In [3]:
wb.get_sheet_names()

['XXXExpense_Calendar',
 "PERSONAL_MONTHLY_BUDGET_SEPT'13",
 "OCT'13",
 "NOV'13",
 "DEC'13",
 "JAN'14",
 "FEB'14",
 "MAR'14",
 "APR'14",
 "MAY'14",
 "JUN'14_",
 "JUL'14",
 "AUG'14",
 "SEPT'14",
 "OCT'14",
 "NOV'14",
 "DEC'14",
 "JAN'15",
 "FEB'15",
 "MARCH'15",
 "APR'15",
 "MAY'15",
 "JUNE'15",
 "JULY'15",
 "AUG'15",
 "SEPT'15",
 "OCT'15",
 "NOV'15",
 "DEC'15",
 'Weekly_Monitor',
 'Monthly_Statement',
 'XXXBank_Sheet',
 'Sheet1']

In [95]:
sheet = wb.get_sheet_by_name('XXXBank_Sheet')

In [5]:
sheet

<Worksheet "XXXBank_Sheet">

In [9]:
sheet['B16'].value

u'OCBC'

In [11]:
type(sheet['B15'].value)

NoneType

In [12]:
print sheet['B15'].value

None


In [24]:
type(sheet['A42'].value)

datetime.datetime

In [25]:
sheet['A42'].value

datetime.datetime(2014, 5, 23, 0, 0)

In [31]:
print sheet['A43'].value, sheet['E43'].value, sheet['M43'].value

2014-05-25 00:00:00 2014-06-04 00:00:00 2014-11-16 00:00:00


In [27]:
print sheet['A42'].value.tzinfo

None


In [28]:
import datetime
import pytz

tz =  pytz.timezone('Asia/Singapore')

if not sheet['A42'].value.tzinfo:
  _when = tz.localize(sheet['A42'].value)
print _when

2014-05-23 00:00:00+08:00


In [32]:
sheet['E53'].value

datetime.datetime(2014, 6, 17, 0, 0)

In [49]:
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Style, Color

In [39]:
wb = openpyxl.Workbook()
sheet = wb.get_sheet_by_name('Sheet')
fontObj1 = Font(name='Times New Roman', bold=True)
styleObj1 = Style(font=fontObj1)
sheet['A1'].style=styleObj1
sheet['A1'] = 'Bold Times New Roman'

fontObj2 = Font(size=24, italic=True)
styleObj2 = Style(font=fontObj2)
sheet['B3'].style=styleObj2
sheet['B3'] = '24 pt Italic'

In [51]:
wb.save('test_py_styles.xlsx')

In [2]:
wb = openpyxl.load_workbook('test_py_styles.xlsx')

In [80]:
wb_data = openpyxl.load_workbook('test_py_styles.xlsx', data_only = True)

In [29]:
sheet = wb.get_sheet_by_name('Sheet')

In [30]:
sheet_data = wb_data.get_sheet_by_name('Sheet')

<Worksheet "Sheet">

In [18]:
new_work_sheet = wb.create_sheet()

In [91]:
sheet['B3'].style = Style(fill=PatternFill(patternType='solid', fgColor=Color('FFFFFF00')),font=fontObj2)

In [ ]:
sheet['B3'].style = Style(fill=PatternFill(patternType='solid', fgColor=Color(rgb_str)),font=fontObj2)

In [92]:
sheet['B3'].style

Style(font=Font(i=True, sz=24.0, color=Color(indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), fill=PatternFill(patternType='solid', fgColor=Color(rgb='FFFFFF00', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), number_format='General')

In [64]:
import struct
rgb = (200,200,50)
rgb_str = struct.pack('BBB',*rgb).encode('hex')

In [83]:
print sheet['B42'].value, sheet['C42'].value

10381.15 EZ Link Ve


In [84]:
type(sheet['C42'].value)

unicode

In [85]:
type(sheet['B42'].value)

float

In [80]:
type(sheet['D18'].value)

float

In [88]:
sheet['F378'].style, type(sheet['F378'].value)

(Style(font=Font(sz=10.0, color=Color(rgb='FF000000', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), fill=PatternFill(patternType='solid', fgColor=Color(rgb='FFFFFF00', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long'), bgColor=Color(rgb='FFFFFF00', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), number_format='[$-4809]General'),
 float)

In [96]:
sheet['B375'].style

Style(font=Font(sz=10.0, color=Color(rgb='FF000000', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), fill=PatternFill(patternType='solid', fgColor=Color(rgb='FFFF00FF', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long'), bgColor=Color(rgb='FFFF00FF', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), number_format='[$-4809]General')

In [100]:
import pprint
pprint.pformat(sheet['B375'].style)

"Style(font=Font(sz=10.0, color=Color(rgb='FF000000', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), fill=PatternFill(patternType='solid', fgColor=Color(rgb='FFFF00FF', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long'), bgColor=Color(rgb='FFFF00FF', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), number_format='[$-4809]General')"

In [101]:
kentut = Style(font=Font(sz=10.0, color=Color(rgb='FF000000', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), fill=PatternFill(patternType='solid', fgColor=Color(rgb='FFFF00FF', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long'), bgColor=Color(rgb='FFFF00FF', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), number_format='[$-4809]General')

SyntaxError: invalid syntax (<ipython-input-101-e8bc13b1fd77>, line 1)

In [102]:
pprint.pprint(sheet['B375'].style)

Style(font=Font(sz=10.0, color=Color(rgb='FF000000', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), fill=PatternFill(patternType='solid', fgColor=Color(rgb='FFFF00FF', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long'), bgColor=Color(rgb='FFFF00FF', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=Value must be type 'long')), number_format='[$-4809]General')


In [16]:
sheet['A5'].value = datetime.datetime(2014, 5, 23, 0, 0)

In [7]:
import datetime

In [15]:
sheet['A4'].value , sheet_data['A4'].value

(datetime.datetime(2014, 5, 23, 0, 0), datetime.datetime(2014, 5, 23, 0, 0))

In [18]:
sheet['A6'] = datetime.datetime(2014, 5, 23, 0, 0)

In [77]:
type(sheet['A4'].value.tzinfo)

NoneType

In [96]:
wb.save('test_py_styles.xlsx')

In [23]:
sheet['A4'].style

C:\Python27\lib\site-packages\openpyxl\styles\styleable.py:111: UserWarning: Use formatting objects such as font directly
  warn("Use formatting objects such as font directly")


Style(font=Font(scheme='minor', color=Color(rgb=Value must be type 'basestring', indexed=Value must be type 'long', auto=Value must be type 'bool', theme=1L, type='theme')), number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy')

In [98]:
sheet.column_dimensions['C'].width

34.5703125

In [24]:
sheet['A5'].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'

In [32]:
sheet['B7'].value , sheet['B8'].value, sheet_data['B7'].value , sheet_data['B8'].value

(3000, '=B7-20', 3000, 2980)

In [33]:
sheet['B9'].value = '=B8-20'

In [3]:
import re

def valid_date_DBS(datestring):
        try:
                mat=re.match('^((([0-9])|([0-2][0-9])|([3][0-1]))\ (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\ \d{4})|(T\+[0-9]+)$', datestring)
                #mat=re.match('^[01][0-9] (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{4}$', datestring)
                if mat is not None:
                        return True
        except: # ValueError:
                pass
        return False

In [33]:
def get_first(iterable, default=None):
    if iterable:
        for item in iterable:
            return item
    return default

In [5]:
import csv
#f = open('acb5e3dd9227f4b749d58dc810ed7059.P000000011915007.csv', 'rt')
#f = open('dbs_2015_0510_2709.csv', 'rt')
f = open('dbs_2015_1510_0510.csv', 'rt')
old_line = []
filtered_lines = []
saldo_akhir_dbs_from_bank = 0
saldo_ledger_dbs_from_bank = 0
try:
    reader = csv.reader(f)
    for line in reader:
        
        if (len(line) == 2) and line[0] == 'Available Balance:':
            _saldo_dbs = float(line[1].replace(',',''))
            print 'got saldo ', _saldo_dbs
            saldo_akhir_dbs_from_bank = _saldo_dbs
            
        if (len(line) == 2) and line[0] == 'Ledger Balance:':
            _saldo_dbs = float(line[1].replace(',',''))
            print 'got saldo ', _saldo_dbs
            saldo_ledger_dbs_from_bank = _saldo_dbs
        
        if valid_date_DBS(get_first(line)):
            #print old_line
            #old_line = line
            #print line
            filtered_lines.append(line)
        #else:
            #old_line= old_line + line
finally:
    f.close()
    for item in filtered_lines:
        print item

got saldo  1183.73
got saldo  1183.73
['14 Oct 2015', 'D2P', ' 40.09', ' ', 'AXS           20151014223724891', '', '', '']
['14 Oct 2015', 'POS', ' 20.00', ' ', 'TRANSIT LI', '', '', '']
['11 Oct 2015', 'POS', ' 20.95', ' ', "WATSON'S P", '', '', '']
['11 Oct 2015', 'POS', ' 32.96', ' ', 'RE & S ENT', '', '', '']
['09 Oct 2015', 'MST', ' 13.80', ' ', 'PAYPAL *MICROSOFTRE OF 40 29 07OCT', '5548-2740-7025-2266', '', '']
['06 Oct 2015', 'MST', ' 48.60', ' ', 'WWF-DONATION           SI NG 05OCT', '5548-2740-7025-2266', '', '']
['05 Oct 2015', 'POS', ' 30.00', ' ', 'TRANSIT LI', '', '', '']
['05 Oct 2015', 'AWL', ' 100.00', ' ', '70252266', 'PIONEER MRT 2', '', '', '']


In [6]:
import datetime
import pytz
from collections import namedtuple

tz =  pytz.timezone('Asia/Singapore')

from datetime import datetime

from openpyxl.compat import range
row = 9

parsed_dbs = []
tuple_parsed_dbs = namedtuple('ParsedDbs', ['when', 'tval', 'info_string'])

for item in filtered_lines:
    #print get_first(item).split()[0:3]
    day, month, year = get_first(item).split()[0:3]
    _when = datetime.strptime("{yyyy} {Mmm} {dd}".format(yyyy=year, Mmm=month, dd=day), "%Y %b %d")
    if not _when.tzinfo:
      _when = tz.localize(_when)
    info_string = ''
    #for info in item[2:3]:
    try:
        _tval = -float(item[2])
    except:
        _tval = float(item[3])
    
    info_string = item[1] + ' ' + item[4]
    for info in item[5::]:
        info_string = info_string + ' ' + info
    print _when, _tval, info_string
    _item_tupled = tuple_parsed_dbs(_when, _tval, info_string)
    parsed_dbs.append(_item_tupled)
    #parsed_dbs.append([_when, _tval, info_string])
    #sheet.cell(column=1,row=row).value = _tval
    row = row + 1

2015-10-14 00:00:00+08:00 -40.09 D2P AXS           20151014223724891   
2015-10-14 00:00:00+08:00 -20.0 POS TRANSIT LI   
2015-10-11 00:00:00+08:00 -20.95 POS WATSON'S P   
2015-10-11 00:00:00+08:00 -32.96 POS RE & S ENT   
2015-10-09 00:00:00+08:00 -13.8 MST PAYPAL *MICROSOFTRE OF 40 29 07OCT 5548-2740-7025-2266  
2015-10-06 00:00:00+08:00 -48.6 MST WWF-DONATION           SI NG 05OCT 5548-2740-7025-2266  
2015-10-05 00:00:00+08:00 -30.0 POS TRANSIT LI   
2015-10-05 00:00:00+08:00 -100.0 AWL 70252266 PIONEER MRT 2   


In [18]:
print parsed_dbs

[ParsedDbs(when=datetime.datetime(2015, 9, 20, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), tval=-89.0, info_string='POS COURTS   '), ParsedDbs(when=datetime.datetime(2015, 9, 20, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), tval=-19.35, info_string='POS NTUC FP   '), ParsedDbs(when=datetime.datetime(2015, 9, 18, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), tval=-43.22, info_string='D2P AXS           20150918205056730   '), ParsedDbs(when=datetime.datetime(2015, 9, 18, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), tval=-100.0, info_string='AWL 70252266 PIONEER MRT 2   '), ParsedDbs(when=datetime.datetime(2015, 9, 17, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), tval=-20.0, info_string='POS TRANSIT LI   '), ParsedDbs(when=datetime.datetime(2015, 9, 17, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), tval=1000.0, info_string='ICT Transferred from OCBC Z621611509170929251557150917092925 OTHR '), ParsedDb

In [95]:
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Style, Color
from openpyxl.cell import coordinate_from_string, column_index_from_string, get_column_letter

row = 9
index_row = 0
saldo = sheet_data.cell(column=2,row=row-1).value 
while sheet.cell(column=1,row=row).value:
    sheet.cell(column=2,row=row).value = '=' + get_column_letter(2) + str(row-1) + '+' + get_column_letter(1) + str(row)
    _cell = sheet.cell(column=3,row=row)
    #_cell = parsed_dbs[index_row][0]
    sheet[get_column_letter(3) + str(row)] = parsed_dbs[index_row][0]
    #_cell.style.number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    sheet[get_column_letter(3) + str(row)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    #saldo = saldo - sheet_data.cell(column=1,row=row).value 
    sheet[get_column_letter(4) + str(row)].value = parsed_dbs[index_row][2]
    saldo = saldo + parsed_dbs[index_row][1]
    row=row+1
    index_row = index_row + 1

print saldo

3274.81


In [5]:
import datetime
import pytz
tz =  pytz.timezone('Asia/Singapore')
d1 = datetime.datetime(2015, 9, 14, 12, 8, 15, 618000, tzinfo=tz)
d2 = datetime.datetime(2015, 9, 14, 11, 8, 15, 618000, tzinfo=tz)
diff = d2 - d1
print diff.days
print d1.replace(hour=0, minute=0, second=0, microsecond=0) == d2.replace(hour=0, minute=0, second=0, microsecond=0)
print (d1.month == d1.month) and (d1.day == d1.day) and (d1.year == d1.year)

-1
True
True


In [86]:
float("123,456.908".replace(',',''))

123456.908

In [87]:
'DBS Savings Plus Account 020-4-069022' == 'DBS Savings Plus Account 020-4-069022'

True

In [90]:
print 1 == 1
print 1 is 1

True
True


In [91]:
'COURTS   ' == parsed_dbs[0][2]

True

In [30]:
import datetime
import re

def valid_date(datestring):
        try:
                mat=re.match('(\d{1,2})[/.-](\d{1,2})[/.-](\d{4})$', datestring)
                if mat is not None:
                        datetime.datetime(*(map(int, mat.groups()[-1::-1])))
                        return True
        except: # ValueError:
                pass
        return False

* Need to change width column for date , and may also for description
* Create table (at different sheet or file) for reporting, and then can copy paste to the main book
* Develop algo for processing
  * Just do comparison date and tval.
  * Report
  * May be develop interface to select which one is verified (optional)

In [37]:
#f = open('TransactionHistory_20150920092712.csv', 'rt')
#f = open('TransactionHistory_2015_0510_2709.csv', 'rt')
f = open('TransactionHistory_2015_1510_0510.csv', 'rt')
old_line = []
filtered_lines = []
g_first = True

saldo_akhir_ocbc_from_bank = 0
saldo_ledger_ocbc_from_bank = 0

try:
    reader = csv.reader(f)
    for line in reader:
        
        if (len(line) == 2) and line[0] == 'Available ':
            _saldo_ocbc = float(line[1].replace(',',''))
            print 'got saldo ', _saldo_ocbc
            saldo_akhir_ocbc_from_bank = _saldo_ocbc
        
        if (len(line) == 2) and line[0] == 'Balance':
            _saldo_ocbc = float(line[1].replace(',',''))
            print 'got saldo ', _saldo_ocbc
            saldo_ledger_ocbc_from_bank = _saldo_ocbc
        #print line, get_first(line), valid_date(get_first(line))
        if valid_date(get_first(line)):
            #print old_line
            if (False == g_first):
                filtered_lines.append(old_line)
            else:
                g_first = False    
            old_line = line
        else:
            old_line= old_line + line
    filtered_lines.append(old_line)            
finally:
    f.close()
    for item in filtered_lines:
        print item

got saldo  8090.01
got saldo  8090.01
['15/10/2015', '15/10/2015', 'FAST PAYMENT', '962.00', '', '', '', 'UOVB 3693749407 OTHR Uncle Jeff Transferred from OCBC ']
['15/10/2015', '15/10/2015', '3RD PTY TRANSFER', '300.00', '', '', '', 'VBS   ']
['14/10/2015', '14/10/2015', 'CASH WITHDRAWAL  ATM', '100.00', '', '', '', '  ']
['14/10/2015', '14/10/2015', 'GIRO - SALARY', '', '4,963.00', '', '', 'INSTITUTE FOR INFOC SALA 000001062250 ']
['13/10/2015', '13/10/2015', 'PURCHASE        NETS', '30.00', '', '', '', 'THANK YOU                PNR  TRANSIT LI ']
['13/10/2015', '13/10/2015', 'PURCHASE        NETS', '18.35', '', '', '', 'COMFORT TAXIS            THANK YOU  COMFORT TR ']
['08/10/2015', '08/10/2015', 'CASH WITHDRAWAL  ATM', '100.00', '', '', '', 'OCBC-31 JURONG WEST ST 63SINGAPORE']


In [39]:
import datetime
import pytz
from collections import namedtuple

tz =  pytz.timezone('Asia/Singapore')

from datetime import datetime

parsed_ocbc = []
tuple_parsed_ocbc = namedtuple('ParsedOcbc', ['when', 'when_value', 'tval', 'info_string'])

for item in filtered_lines:
    #print get_first(item)
    #_when = get_valid_date(get_first(item))
    _when = datetime.strptime(get_first(item), "%d/%m/%Y")
    _when2 = datetime.strptime(item[1], "%d/%m/%Y")
    if not _when.tzinfo:
      _when = tz.localize(_when)
    if not _when2.tzinfo:
      _when2 = tz.localize(_when2)
    info_string = ''
    #for info in item[2::]:
    try:
        _tval = -float(item[3].replace(',',''))
    except:
        _tval = float(item[4].replace(',',''))
    
    if len(item) > 5:
        info_string = item[2] + ' ' + item[5]
        for info in item[6::]:
            info_string = info_string + ' ' + info
    else:
        info_string = item[2]
    print _when , _when2, _tval, info_string
    _item_tupled = tuple_parsed_ocbc(_when , _when2, _tval, info_string)
    parsed_ocbc.append(_item_tupled)

2015-10-15 00:00:00+08:00 2015-10-15 00:00:00+08:00 -962.0 FAST PAYMENT   UOVB 3693749407 OTHR Uncle Jeff Transferred from OCBC 
2015-10-15 00:00:00+08:00 2015-10-15 00:00:00+08:00 -300.0 3RD PTY TRANSFER   VBS   
2015-10-14 00:00:00+08:00 2015-10-14 00:00:00+08:00 -100.0 CASH WITHDRAWAL  ATM     
2015-10-14 00:00:00+08:00 2015-10-14 00:00:00+08:00 4963.0 GIRO - SALARY   INSTITUTE FOR INFOC SALA 000001062250 
2015-10-13 00:00:00+08:00 2015-10-13 00:00:00+08:00 -30.0 PURCHASE        NETS   THANK YOU                PNR  TRANSIT LI 
2015-10-13 00:00:00+08:00 2015-10-13 00:00:00+08:00 -18.35 PURCHASE        NETS   COMFORT TAXIS            THANK YOU  COMFORT TR 
2015-10-08 00:00:00+08:00 2015-10-08 00:00:00+08:00 -100.0 CASH WITHDRAWAL  ATM   OCBC-31 JURONG WEST ST 63SINGAPORE


In [28]:
print parsed_ocbc

[ParsedOcbc(when=datetime.datetime(2015, 9, 18, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), when_value=datetime.datetime(2015, 9, 18, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), tval=-30.0, info_string='PURCHASE        NETS   THANK YOU                CITY HALL  SMRT - CTH '), ParsedOcbc(when=datetime.datetime(2015, 9, 18, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), when_value=datetime.datetime(2015, 9, 18, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), tval=-1010.0, info_string='PURCHASE        NETS   THANK YOU                ROBINSON  SINGPOST '), ParsedOcbc(when=datetime.datetime(2015, 9, 17, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), when_value=datetime.datetime(2015, 9, 17, 0, 0, tzinfo=<DstTzInfo 'Asia/Singapore' SGT+8:00:00 STD>), tval=-970.0, info_string='FAST PAYMENT   Transferred from OCBC UOVB 3693749407 OTHR Uncle Jeff '), ParsedOcbc(when=datetime.datetime(2015, 9, 17, 0, 0, tzinfo=<DstTzInfo 'Asia

In [103]:
for parsed_item in parsed_ocbc:
    print parsed_item.when , parsed_item.when_value, parsed_item.tval, parsed_item.info_string

2015-10-03 00:00:00+08:00 2015-10-03 00:00:00+08:00 -30.0 PURCHASE        NETS   THANK YOU                BBS  TRANSIT LI 
2015-10-03 00:00:00+08:00 2015-10-03 00:00:00+08:00 -100.0 CASH WITHDRAWAL  ATM   OCBC-TIONG BAHRU MRT     SINGAPORE
2015-10-01 00:00:00+08:00 2015-09-30 00:00:00+08:00 0.23 INTEREST CREDIT
2015-10-01 00:00:00+08:00 2015-10-01 00:00:00+08:00 -462.68 IBG GIRO   88854193 *PRUDENTIAL A CO NO COLL 88854193 
2015-10-01 00:00:00+08:00 2015-10-01 00:00:00+08:00 -405.93 IBG GIRO   88854193 *PRUDENTIAL A CO NO COLL 88854193 
2015-09-27 00:00:00+08:00 2015-09-28 00:00:00+08:00 -200.0 CASH WITHDRAWAL  ATM   OCBC-31 JURONG WEST ST 63SINGAPORE
2015-09-25 00:00:00+08:00 2015-09-25 00:00:00+08:00 -16.45 PURCHASE        NETS   COMFORT TAXIS            THANK YOU  COMFORT TR 
2015-09-25 00:00:00+08:00 2015-09-25 00:00:00+08:00 -30.0 PURCHASE        NETS   THANK YOU                BUGIS MRT  SMRT - BGS 
2015-09-22 00:00:00+08:00 2015-09-22 00:00:00+08:00 -100.0 CASH WITHDRAWAL  ATM  

In [78]:
for parsed_item in parsed_dbs:
    print parsed_item.when , parsed_item.tval, parsed_item.info_string

2015-10-05 00:00:00+08:00 -30.0 POS TRANSIT LI   
2015-10-05 00:00:00+08:00 -100.0 AWL 70252266 PIONEER MRT 2   
2015-10-04 00:00:00+08:00 640.0 ITR I-BANK anthony  
2015-10-04 00:00:00+08:00 -9.24 MST PAYPAL *NETFLIX.COM    40 29 02OCT 5548-2740-7025-2266  
2015-09-30 00:00:00+08:00 0.03 INT    
2015-09-30 00:00:00+08:00 -10.0 D2P AXS           20150930202311935   
2015-09-30 00:00:00+08:00 -10.0 D2P AXS           20150930111301542   
2015-09-29 00:00:00+08:00 -24.5 MST GOLDEN VILLAGE MULTIPL SI NG 25SEP 5548-2740-7025-2266  
2015-09-27 00:00:00+08:00 -17.8 POS NTUC FAIRP   
2015-09-27 00:00:00+08:00 -29.74 POS ETERNAL HA   
2015-09-26 00:00:00+08:00 -6.0 D2P AXS           20150926202446564   
2015-09-25 00:00:00+08:00 -30.0 POS TRANSIT LI   
2015-09-24 00:00:00+08:00 -29.9 D2P AXS           20150924162715735   
2015-09-22 00:00:00+08:00 -53.5 MST CATHAY CINEPLEXES PTE  SI NG 19SEP 5548-2740-7025-2266  
2015-09-20 00:00:00+08:00 -89.0 POS COURTS   
2015-09-20 00:00:00+08:00 -19.35 POS

In [87]:
wb.remove_sheet(new_work_sheet)

In [88]:
new_work_sheet = wb.create_sheet()
new_work_sheet.title = "ParsedDbs_wparse_Sept"

In [7]:
new_work_sheet = wb.get_sheet_by_name("ParsedDbs_wparse_Sept")

In [19]:
new_work_sheet = wb.get_sheet_by_name("ParsedOcbc_wparse_Sept")

In [96]:
new_work_sheet = wb.create_sheet()
new_work_sheet.title = "ParsedOcbc_wparse_Sept"

In [8]:
for parsed_item in parsed_dbs:
    if parsed_item.when > datetime(2015, 8, 24, tzinfo=tz):
        print parsed_item.when , parsed_item.tval, parsed_item.info_string

2015-10-14 00:00:00+08:00 -40.09 D2P AXS           20151014223724891   
2015-10-14 00:00:00+08:00 -20.0 POS TRANSIT LI   
2015-10-11 00:00:00+08:00 -20.95 POS WATSON'S P   
2015-10-11 00:00:00+08:00 -32.96 POS RE & S ENT   
2015-10-09 00:00:00+08:00 -13.8 MST PAYPAL *MICROSOFTRE OF 40 29 07OCT 5548-2740-7025-2266  
2015-10-06 00:00:00+08:00 -48.6 MST WWF-DONATION           SI NG 05OCT 5548-2740-7025-2266  
2015-10-05 00:00:00+08:00 -30.0 POS TRANSIT LI   
2015-10-05 00:00:00+08:00 -100.0 AWL 70252266 PIONEER MRT 2   


In [117]:
for parsed_item in reversed(parsed_dbs_filtered_by_date):
    print parsed_item.when , parsed_item.tval, parsed_item.info_string, parsed_item.when < datetime(2015, 10, 4, tzinfo=tz)

2015-09-17 00:00:00+08:00 1000.0 ICT Transferred from OCBC Z621611509170929251557150917092925 OTHR  True
2015-09-17 00:00:00+08:00 -20.0 POS TRANSIT LI    True
2015-09-18 00:00:00+08:00 -100.0 AWL 70252266 PIONEER MRT 2    True
2015-09-18 00:00:00+08:00 -43.22 D2P AXS           20150918205056730    True
2015-09-20 00:00:00+08:00 -19.35 POS NTUC FP    True
2015-09-20 00:00:00+08:00 -89.0 POS COURTS    True
2015-09-22 00:00:00+08:00 -53.5 MST CATHAY CINEPLEXES PTE  SI NG 19SEP 5548-2740-7025-2266   True
2015-09-24 00:00:00+08:00 -29.9 D2P AXS           20150924162715735    True
2015-09-25 00:00:00+08:00 -30.0 POS TRANSIT LI    True
2015-09-26 00:00:00+08:00 -6.0 D2P AXS           20150926202446564    True
2015-09-27 00:00:00+08:00 -29.74 POS ETERNAL HA    True
2015-09-27 00:00:00+08:00 -17.8 POS NTUC FAIRP    True
2015-09-29 00:00:00+08:00 -24.5 MST GOLDEN VILLAGE MULTIPL SI NG 25SEP 5548-2740-7025-2266   True
2015-09-30 00:00:00+08:00 -10.0 D2P AXS           20150930111301542    True
20

In [82]:
parsed_dbs_filtered_by_date = [x for x in parsed_dbs if x.when > datetime(2015, 8, 21, tzinfo=tz)]

from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Style, Color
from openpyxl.cell import coordinate_from_string, column_index_from_string, get_column_letter

row = 2
index_row = 0
saldo_awal = 342.35
saldo_akhir = saldo_awal
sheet = new_work_sheet

sheet.cell(column=2,row=1).value = saldo_awal

for parsed_item in reversed(parsed_dbs_filtered_by_date):
    sheet.cell(column=2,row=row).value = '=' + get_column_letter(2) + str(row-1) + '+' + get_column_letter(3) + str(row)
    #_cell = sheet.cell(column=3,row=row)
    #_cell = parsed_dbs[index_row][0]
    sheet[get_column_letter(1) + str(row)] = parsed_item.when
    #_cell.style.number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    sheet[get_column_letter(1) + str(row)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    #saldo = saldo - sheet_data.cell(column=1,row=row).value 
    sheet[get_column_letter(3) + str(row)].value = parsed_item.tval
    sheet[get_column_letter(4) + str(row)].value = parsed_item.info_string
    
    saldo_akhir = saldo_akhir + parsed_item.tval
    row=row+1
    index_row = index_row + 1

sheet.column_dimensions['A'].width = 35

print saldo_akhir, saldo_akhir_dbs_from_bank, saldo_ledger_dbs_from_bank
if 0.01 < abs(saldo_akhir - saldo_akhir_dbs_from_bank):
    print "WARNING: saldo not tally!"
    if 0.01 > abs(saldo_akhir - saldo_ledger_dbs_from_bank):
        print "INFERENCE: Possible unsettled transaction in Debit Card detected..."
    print "CHECK: \n- if you got any unsettled transaction (e.g. debit cards) \n- or transaction date cut-off is wrong \n- or saldo is incorrect"
    print '(Total) Difference is ', -saldo_akhir + saldo_akhir_dbs_from_bank
print -saldo_akhir + saldo_akhir_dbs_from_bank, -saldo_akhir + saldo_ledger_dbs_from_bank

1360.13 1311.53 1360.13
INFERENCE: Possible unsettled transaction in Debit Card detected...
CHECK: 
- if you got any unsettled transaction (e.g. debit cards) 
- or transaction date cut-off is wrong 
- or saldo is incorrect
(Total) Difference is  -48.6
-48.6 0.0


In [15]:
wb.save('test_py_styles.xlsx')

snippet for loading yaml

In [9]:
from collections import namedtuple
import yaml

tuple_wlist = namedtuple('WList', ['word_list', 'description', 'category'])
fo = open('expense_parsing_wordlist.yaml')
fo_str = fo.read()
fo.close()
wlist = yaml.load(fo_str)

wlist_dbs = []
wlist_ocbc = []

for mat in wlist['DBS']:
    mat_tupled = tuple_wlist(**mat)
    wlist_dbs.append(mat_tupled)
for mat in wlist['OCBC']:
    mat_tupled = tuple_wlist(**mat)
    wlist_ocbc.append(mat_tupled)

In [10]:
import re
def parseCategory(str,wlist):
    for entry in wlist:
        for word in entry.word_list:
            if re.search(word,str):
                return entry.category, entry.description

## DBS Combined with pyaml for parsing categories

In [92]:
parsed_dbs_filtered_by_date = [x for x in parsed_dbs if x.when > datetime(2015, 8, 24, tzinfo=tz)]
#parsed_dbs_filtered_by_date = parsed_dbs

from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Style, Color
from openpyxl.cell import coordinate_from_string, column_index_from_string, get_column_letter

row = 23
index_row = 0
saldo_awal = 1490.13#342.35 # 911.05 # 841.14
saldo_akhir = saldo_awal
sheet = new_work_sheet

sheet.cell(column=2,row=1).value = saldo_awal

for parsed_item in reversed(parsed_dbs_filtered_by_date):
    sheet.cell(column=2,row=row).value = '=' + get_column_letter(2) + str(row-1) + '+' + get_column_letter(3) + str(row)
    #_cell = sheet.cell(column=3,row=row)
    #_cell = parsed_dbs[index_row][0]
    sheet[get_column_letter(1) + str(row)] = parsed_item.when
    #_cell.style.number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    sheet[get_column_letter(1) + str(row)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    #saldo = saldo - sheet_data.cell(column=1,row=row).value 
    sheet[get_column_letter(3) + str(row)].value = parsed_item.tval
    sheet[get_column_letter(4) + str(row)].value = parsed_item.info_string
    
    category = parseCategory(parsed_item.info_string, wlist_dbs)
    if category:
        if len(category[0])>1:
            sheet[get_column_letter(5) + str(row)].value = 'MULTIPLE: '+';'.join(category[0])
        else:
            sheet[get_column_letter(5) + str(row)].value = ';'.join(category[0])
        sheet[get_column_letter(6) + str(row)].value = ';'.join(category[1])
    else:
        sheet[get_column_letter(5) + str(row)].value = 'NONE'
    
    saldo_akhir = saldo_akhir + parsed_item.tval
    row=row+1
    index_row = index_row + 1

sheet.column_dimensions['A'].width = 35

print saldo_akhir, saldo_akhir_dbs_from_bank, saldo_ledger_dbs_from_bank
if 0.01 < abs(saldo_akhir - saldo_akhir_dbs_from_bank):
    print "WARNING: saldo not tally!"
    if 0.01 > abs(saldo_akhir - saldo_ledger_dbs_from_bank):
        print "INFERENCE: Possible unsettled transaction in Debit Card detected..."
    print "CHECK: \n- if you got any unsettled transaction (e.g. debit cards) \n- or transaction date cut-off is wrong \n- or saldo is incorrect"
    print '(Total) Difference is ', -saldo_akhir + saldo_akhir_dbs_from_bank
print -saldo_akhir + saldo_akhir_dbs_from_bank, -saldo_akhir + saldo_ledger_dbs_from_bank

1360.13 1311.53 1360.13
INFERENCE: Possible unsettled transaction in Debit Card detected...
CHECK: 
- if you got any unsettled transaction (e.g. debit cards) 
- or transaction date cut-off is wrong 
- or saldo is incorrect
(Total) Difference is  -48.6
-48.6 0.0


## Revised DBS for parsing, with different category with different view

In [13]:
#parsed_dbs_filtered_by_date = [x for x in parsed_dbs if x.when > datetime(2015, 8, 24, tzinfo=tz)]
parsed_dbs_filtered_by_date = parsed_dbs
#parsed_dbs_filtered_by_date = [x for x in parsed_dbs if x.when < datetime(2015, 10, 5, tzinfo=tz)]
#parsed_dbs_filtered_by_date = [x for x in parsed_dbs if x.when.day < 5 or x.when.month < 10]

from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Style, Color
from openpyxl.cell import coordinate_from_string, column_index_from_string, get_column_letter

row = 23
index_row = 0
saldo_awal = 1490.13#342.35 # 911.05 # 841.14
saldo_akhir = saldo_awal
sheet = new_work_sheet

category_set_weekly_dbs  = {}
category_set_monthly_dbs = {}
category_set_detailed_monthly_dbs = {}

sheet.cell(column=2,row=1).value = saldo_awal

for parsed_item in reversed(parsed_dbs_filtered_by_date):
    sheet.cell(column=2,row=row).value = '=' + get_column_letter(2) + str(row-1) + '+' + get_column_letter(4) + str(row)
    #_cell = sheet.cell(column=3,row=row)
    #_cell = parsed_dbs[index_row][0]
    sheet[get_column_letter(1) + str(row)] = parsed_item.when
    #_cell.style.number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    sheet[get_column_letter(1) + str(row)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    #saldo = saldo - sheet_data.cell(column=1,row=row).value 
    sheet[get_column_letter(4) + str(row)].value = parsed_item.tval
    sheet[get_column_letter(5) + str(row)].value = parsed_item.info_string
    
    category = parseCategory(parsed_item.info_string, wlist_dbs)
    #if category:
    #    if len(category[0])>1:
    #        sheet[get_column_letter(5) + str(row)].value = 'MULTIPLE: '+';'.join(category[0])
    #    else:
    #        sheet[get_column_letter(5) + str(row)].value = ';'.join(category[0])
    #    sheet[get_column_letter(6) + str(row)].value = ';'.join(category[1])
    #else:
    #    sheet[get_column_letter(5) + str(row)].value = 'NONE'
    
    if category:
        sheet[get_column_letter(6) + str(row)].value = category[0]['detailed_monthly']
        sheet[get_column_letter(7) + str(row)].value = category[0]['monthly']
        sheet[get_column_letter(8) + str(row)].value = category[0]['weekly']
        sheet[get_column_letter(3) + str(row)].value = ';'.join(category[1])
        category_set_monthly_dbs[category[0]['monthly']] = category_set_monthly_dbs.get(category[0]['monthly'],0)+parsed_item.tval
        category_set_weekly_dbs[category[0]['weekly']] = category_set_weekly_dbs.get(category[0]['weekly'],0)+parsed_item.tval
        category_set_detailed_monthly_dbs[category[0]['detailed_monthly']] = category_set_detailed_monthly_dbs.get(category[0]['detailed_monthly'],0)+parsed_item.tval
    else:
        sheet[get_column_letter(6) + str(row)].value = 'PLEASE_DESC'
        sheet[get_column_letter(7) + str(row)].value = 'PLEASE_DESC'
        sheet[get_column_letter(8) + str(row)].value = 'PLEASE_DESC'
    
    saldo_akhir = saldo_akhir + parsed_item.tval
    row=row+1
    index_row = index_row + 1

sheet.column_dimensions['A'].width = 35

print saldo_akhir, saldo_akhir_dbs_from_bank, saldo_ledger_dbs_from_bank
if 0.01 < abs(saldo_akhir - saldo_akhir_dbs_from_bank):
    print "WARNING: saldo not tally!"
    if 0.01 > abs(saldo_akhir - saldo_ledger_dbs_from_bank):
        print "INFERENCE: Possible unsettled transaction in Debit Card detected..."
    print "CHECK: \n- if you got any unsettled transaction (e.g. debit cards) \n- or transaction date cut-off is wrong \n- or saldo is incorrect"
    print '(Total) Difference is ', -saldo_akhir + saldo_akhir_dbs_from_bank
print -saldo_akhir + saldo_akhir_dbs_from_bank, -saldo_akhir + saldo_ledger_dbs_from_bank

1183.73 1183.73 1183.73
-2.27373675443e-13 -2.27373675443e-13


In [14]:
print category_set_monthly_dbs, sum(category_set_monthly_dbs.values())
print category_set_weekly_dbs, sum(category_set_weekly_dbs.values())
print category_set_detailed_monthly_dbs, sum(category_set_detailed_monthly_dbs.values())

{'giving': -48.6, 'comm': -40.09, 'makan_atm': -100.0, 'recurr': -13.8, 'transport': -50.0} -252.49
{'NONE': -162.4, 'comm': -40.09, 'transport': -50.0} -252.49
{'giving': -48.6, 'comm': -40.09, 'NONE': -100.0, 'online_subscription': -13.8, 'transport': -50.0} -252.49


In [59]:
for parsed_item in parsed_ocbc:
    if parsed_item.when > datetime(2015, 8, 23, tzinfo=tz):
        print parsed_item.when , parsed_item.when_value, parsed_item.tval, parsed_item.info_string

2015-09-09 00:00:00+08:00 2015-09-09 00:00:00+08:00 -100.0 CASH WITHDRAWAL  ATM   OCBC-31 JURONG WEST ST 63SINGAPORE
2015-09-09 00:00:00+08:00 2015-09-09 00:00:00+08:00 -30.0 PURCHASE        NETS   THANK YOU                ORCHARD  SMRT-ORC 
2015-09-08 00:00:00+08:00 2015-09-08 00:00:00+08:00 -24.6 PURCHASE        NETS   COMFORT TAXIS            THANK YOU  COMFORT TR 
2015-09-07 00:00:00+08:00 2015-09-07 00:00:00+08:00 -37.1 PURCHASE        NETS   THANK YOU                BRAS BASA  0481431MLC 
2015-09-02 00:00:00+08:00 2015-09-02 00:00:00+08:00 -30.0 PURCHASE        NETS   THANK YOU                PNR  TRANSIT LI 
2015-09-01 00:00:00+08:00 2015-08-31 00:00:00+08:00 0.25 INTEREST CREDIT
2015-09-01 00:00:00+08:00 2015-09-01 00:00:00+08:00 -462.68 IBG GIRO   88854193 *PRUDENTIAL A CO NO COLL 88854193 
2015-09-01 00:00:00+08:00 2015-09-01 00:00:00+08:00 -405.93 IBG GIRO   88854193 *PRUDENTIAL A CO NO COLL 88854193 
2015-08-28 00:00:00+08:00 2015-08-28 00:00:00+08:00 -100.0 CASH WITHDRAWAL

In [49]:
new_work_sheet = wb.create_sheet()
new_work_sheet.title = "ParsedOcbc_wparse"

In [60]:
parsed_ocbc_filtered_by_date = [x for x in parsed_ocbc if x.when > datetime(2015, 8, 23, tzinfo=tz)]

In [61]:
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Style, Color
from openpyxl.cell import coordinate_from_string, column_index_from_string, get_column_letter

row = 2
index_row = 0
saldo_awal = 5632.2
saldo_akhir = saldo_awal
sheet = new_work_sheet

sheet.cell(column=2,row=1).value = saldo_awal

for parsed_item in reversed(parsed_ocbc_filtered_by_date):
    sheet.cell(column=2,row=row).value = '=' + get_column_letter(2) + str(row-1) + '+' + get_column_letter(3) + str(row)
    #_cell = sheet.cell(column=3,row=row)
    #_cell = parsed_dbs[index_row][0]
    sheet[get_column_letter(1) + str(row)] = parsed_item.when
    #_cell.style.number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    sheet[get_column_letter(1) + str(row)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    #saldo = saldo - sheet_data.cell(column=1,row=row).value 
    sheet[get_column_letter(3) + str(row)].value = parsed_item.tval
    sheet[get_column_letter(4) + str(row)].value = parsed_item.info_string
    
    saldo_akhir = saldo_akhir + parsed_item.tval
    row=row+1
    index_row = index_row + 1

sheet.column_dimensions['A'].width = 35

print saldo_akhir, saldo_akhir_ocbc_from_bank, saldo_ledger_ocbc_from_bank
if 0.01 < abs(saldo_akhir - saldo_akhir_ocbc_from_bank):
    print "WARNING: saldo not tally!"
    if 0.01 > abs(saldo_akhir - saldo_ledger_ocbc_from_bank):
        print "INFERENCE: Possible unsettled transaction in Debit Card detected..."
    print "CHECK: \n- if you got any unsettled transaction (e.g. debit cards) \n- or transaction date cut-off is wrong \n- or saldo is incorrect"
    print '(Total) Difference is ', -saldo_akhir + saldo_akhir_ocbc_from_bank
else:
    print 'OK, saldo tally.'
print -saldo_akhir + saldo_akhir_ocbc_from_bank, -saldo_akhir + saldo_ledger_ocbc_from_bank

4342.14 4767.36 4767.36
CHECK: 
- if you got any unsettled transaction (e.g. debit cards) 
- or transaction date cut-off is wrong 
- or saldo is incorrect
(Total) Difference is  425.22
425.22 425.22


## OCBC Combined with pyaml for parsing categories

In [40]:
#parsed_ocbc_filtered_by_date = [x for x in parsed_ocbc if x.when > datetime(2015, 8, 23, tzinfo=tz)]
#parsed_ocbc_filtered_by_date = [x for x in parsed_ocbc if x.when < datetime(2015, 10, 5, tzinfo=tz)]
#parsed_ocbc_filtered_by_date = [x for x in parsed_ocbc if x.when.day < 5 or x.when.month < 10]
parsed_ocbc_filtered_by_date = parsed_ocbc
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Style, Color
from openpyxl.cell import coordinate_from_string, column_index_from_string, get_column_letter

row = 13 #2
index_row = 0
saldo_awal = 4637.36 #9425.14 # 4342.14 # 5632.2
saldo_akhir = saldo_awal
sheet = new_work_sheet

category_set_weekly_ocbc  = {}
category_set_monthly_ocbc = {}
category_set_detailed_monthly_ocbc = {}

sheet.cell(column=2,row=1).value = saldo_awal

for parsed_item in reversed(parsed_ocbc_filtered_by_date):
    sheet.cell(column=2,row=row).value = '=' + get_column_letter(2) + str(row-1) + '+' + get_column_letter(4) + str(row)
    #_cell = sheet.cell(column=3,row=row)
    #_cell = parsed_dbs[index_row][0]
    sheet[get_column_letter(1) + str(row)] = parsed_item.when
    #_cell.style.number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    sheet[get_column_letter(1) + str(row)].number_format='[$-F800]dddd\\,\\ mmmm\\ dd\\,\\ yyyy'
    #saldo = saldo - sheet_data.cell(column=1,row=row).value 
    sheet[get_column_letter(4) + str(row)].value = parsed_item.tval
    sheet[get_column_letter(5) + str(row)].value = parsed_item.info_string
    
    category = parseCategory(parsed_item.info_string, wlist_ocbc)
    #if category:
    #    if len(category[0])>1:
    #        sheet[get_column_letter(5) + str(row)].value = 'MULTIPLE: '+';'.join(category[0])
    #    else:
    #        sheet[get_column_letter(5) + str(row)].value = ';'.join(category[0])
    #    sheet[get_column_letter(6) + str(row)].value = ';'.join(category[1])
    #else:
    #    sheet[get_column_letter(5) + str(row)].value = 'NONE'
    
    if category:
        sheet[get_column_letter(6) + str(row)].value = category[0]['detailed_monthly']
        sheet[get_column_letter(7) + str(row)].value = category[0]['monthly']
        sheet[get_column_letter(8) + str(row)].value = category[0]['weekly']
        sheet[get_column_letter(3) + str(row)].value = ';'.join(category[1])
        category_set_monthly_ocbc[category[0]['monthly']] = category_set_monthly_ocbc.get(category[0]['monthly'],0)+parsed_item.tval
        category_set_weekly_ocbc[category[0]['weekly']] = category_set_weekly_ocbc.get(category[0]['weekly'],0)+parsed_item.tval
        category_set_detailed_monthly_ocbc[category[0]['detailed_monthly']] = category_set_detailed_monthly_ocbc.get(category[0]['detailed_monthly'],0)+parsed_item.tval
    else:
        sheet[get_column_letter(6) + str(row)].value = 'PLEASE_DESC'
        sheet[get_column_letter(7) + str(row)].value = 'PLEASE_DESC'
        sheet[get_column_letter(8) + str(row)].value = 'PLEASE_DESC'
    
    saldo_akhir = saldo_akhir + parsed_item.tval
    row=row+1
    index_row = index_row + 1

sheet.column_dimensions['A'].width = 35

print saldo_akhir, saldo_akhir_ocbc_from_bank, saldo_ledger_ocbc_from_bank
if 0.01 < abs(saldo_akhir - saldo_akhir_ocbc_from_bank):
    print "WARNING: saldo not tally!"
    if 0.01 > abs(saldo_akhir - saldo_ledger_ocbc_from_bank):
        print "INFERENCE: Possible unsettled transaction in Debit Card detected..."
    print "CHECK: \n- if you got any unsettled transaction (e.g. debit cards) \n- or transaction date cut-off is wrong \n- or saldo is incorrect"
    print '(Total) Difference is ', -saldo_akhir + saldo_akhir_ocbc_from_bank
else:
    print 'OK, saldo tally.'
print -saldo_akhir + saldo_akhir_ocbc_from_bank, -saldo_akhir + saldo_ledger_ocbc_from_bank

8090.01 8090.01 8090.01
OK, saldo tally.
1.81898940355e-12 1.81898940355e-12


In [41]:
wb.save('test_py_styles.xlsx')

In [42]:
print category_set_monthly_ocbc, sum(category_set_monthly_ocbc.values())
print category_set_weekly_ocbc, sum(category_set_weekly_ocbc.values())
print category_set_detailed_monthly_ocbc, sum(category_set_detailed_monthly_ocbc.values())

{'makan_atm': -200.0, 'rent': -962.0, 'transport': -48.35, 'income': 4963.0} 3752.65
{'NONE': 3801.0, 'transport': -48.35} 3752.65
{'taxi': -18.35, 'NONE': -200.0, 'income': 4963.0, 'rent': -962.0, 'ezlink': -30.0} 3752.65
